# Productions @ Event-level

Ultimately, our objective is to conduct a comparative analysis of the data and simulations.
<br>
Therefore, it is important to have both information with the same structure at event-level.

In [1]:
import sys
sys.path.append('/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/libs')

import crudo
import fit_functions as ff
import glob
from   iminuit import Minuit
from   iminuit.cost import LeastSquares
from   invisible_cities.reco.corrections import read_maps
from   invisible_cities.reco.corrections import apply_all_correction
from   invisible_cities.types.symbols    import NormStrategy
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import plotting_tools as pt
import tables as tb
from   scipy import integrate
from   scipy.optimize import curve_fit
from   scipy.stats    import linregress

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Preliminary

In [2]:
# ----- Important Directories ----- #
data_dir   = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/sophronia/'                           # Runs per ldc
icaros_dir = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/icaros/'                              # Kr maps per run
MC_dir     = '/lustre/ific.uv.es/prj/gl/neutrinos/NEXT/MC/NEXT100/Radiogenics/LPR/IC_v2.3.1/NEXUS_v7_08_01/'    # MC directory
output_dir = '/lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Backgrounds/h5/'                      # Output directory for the h5 files

# Paths of the tables inside the HDF5 file. These are the keys used to access the datasets
dorothea_key  = '/DST/Events'
sophronia_key = '/RECO/Events'

# Data Production

In [3]:
# List of the low-background runs
runs_info = {
                # Condition: castle = closed & RAS = on #
                # Jun 2025
                15502: {"duration": 85477, "OK": 28287, "LOST": 9564  , "proc_eff": 1.0   },
                # 15504: {"duration": 85194, "OK": 28297, "LOST": 9537  , "proc_eff": 0.9999},
                # 15505: {"duration": 86517, "OK": 28632, "LOST": 9725  , "proc_eff": 0.9999},
                # 15506: {"duration": 84841, "OK": 28438, "LOST": 9603  , "proc_eff": 0.9999},
                # 15507: {"duration": 55740, "OK": 18569, "LOST": 6149  , "proc_eff": 0.9999},
                # 15514: {"duration": 59207, "OK": 20054, "LOST": 6646  , "proc_eff": 0.9999},
                # 15519: {"duration": 34045, "OK": 11420, "LOST": 3646  , "proc_eff": 0.9998},
                # 15520: {"duration": 85170, "OK": 28050, "LOST": 9130  , "proc_eff": 1.0   },
                # 15521: {"duration": 85388, "OK": 28396, "LOST": 8831  , "proc_eff": 1.0   },
                # 15527: {"duration": 69725, "OK": 23560, "LOST": 7411  , "proc_eff": 1.0   },
                # 15528: {"duration": 41361, "OK": 13460, "LOST": 4426  , "proc_eff": 1.0   },
                # 15535: {"duration": 84904, "OK": 28799, "LOST": 9156  , "proc_eff": 0.9998},
                # 15539: {"duration": 56567, "OK": 15618, "LOST": 9586  , "proc_eff": 0.9997},
                # 15540: {"duration": 67663, "OK": 22526, "LOST": 7066  , "proc_eff": 0.9993},
                # 15541: {"duration": 86630, "OK": 29124, "LOST": 9333  , "proc_eff": 1.0   },
                # 15542: {"duration": 87915, "OK": 29717, "LOST": 9274  , "proc_eff": 1.0   },
                # 15543: {"duration": 86570, "OK": 29160, "LOST": 9123  , "proc_eff": 1.0   },
                # 15544: {"duration": 86566, "OK": 29498, "LOST": 9029  , "proc_eff": 1.0   },
                # 15545: {"duration": 85892, "OK": 29437, "LOST": 8877  , "proc_eff": 1.0   },
                # 15546: {"duration": 84822, "OK": 28663, "LOST": 8704  , "proc_eff": 1.0   },
                # 15547: {"duration": 71594, "OK": 24549, "LOST": 7421  , "proc_eff": 1.0   },
                # 15557: {"duration": 66088, "OK": 22273, "LOST": 6929  , "proc_eff": 1.0   },
            }

## Processing

In [ ]:
# Global
V_drift = 0.865     # in [mm/μs]

# Setting up the filter of spurious hits
filter_config = {'distance': [15., 15.], 'nhit': 3, 'variables': ['E_corr']}
NO_hits_sp = crudo.drop_isolated_clusters(**filter_config)

# ----- Information for S1e Correction ----- #
# Cathode temporal position
DT_stop = 1372.2543             # in [μs]
# Fit values
CV_fit = [0.62, 869.87]         # [slope, intercept]

In [ ]:
# Efficiency counters
sophronia_events  = 0
Znegative_events  = 0
final_reco_events = 0

# Store all the processed dfs
all_processed_dfs = []

# ----- Run Loop ----- #
for run_id, entry in runs_info.items():

    print(f"--- Processing run {run_id} ---")

    # ----- Krypton Map ----- #
    try:
        kr_file = f'run_{run_id}.v2.3.1.20250512.Kr.map.h5'
        kr_path = os.path.join(icaros_dir, kr_file)
        # Read map and create correction function
        cmap = read_maps(kr_path)
        corr_func = apply_all_correction(cmap, apply_temp=False, norm_strat=NormStrategy.kr)
    except FileNotFoundError:
        print(f"Kr map for run {run_id} not found. Skipping...")
        continue

    # Run counter
    reco_evts = 0
    events = 0

    # ----- LDC Loop ----- #
    for ldc in range(1, 8):

        # Load the HDF5 file
        h5_path = os.path.join(data_dir, f'run_{run_id}_ldc{ldc}_trg2_sophronia.h5')

        # ----- Sophronia ----- #
        df_sophronia = pd.DataFrame()
        try:
            df_sophronia = pd.read_hdf(h5_path, key=sophronia_key, columns=['event', 'time', 'npeak', 'X', 'Y', 'Z', 'E'])
            print(f"  After reco chain: {df.sophronia.event.nunique()} events")
            sophronia_events += df_sophronia.event.nunique()
        except KeyError:
            print(f"Key {sophronia_key} not found in {h5_path}. Skipping...")
            continue

        # ----- Energy Correction ----- #
        df_sophronia['corr_factor'] = corr_func(df_sophronia.X, df_sophronia.Y, df_sophronia.Z, df_sophronia.time)      # In data, DT = Z
        df_sophronia['E_corr'] = df_sophronia['E'] * df_sophronia['corr_factor']

        # Set hits with NaN or negative energy to 0
        df_sophronia['E_corr'] = np.where(
                                            pd.notna(df_sophronia['E_corr']) & (df_sophronia['E_corr'] > 0),        # Condition
                                            df_sophronia['E_corr'],                                                 # Value if condition is True
                                            0                                                                       # Value if condition is False   
        )

        # Drop events with negative Z
        bad_evt_ids = df_sophronia.loc[df_sophronia['Z'] < 0, 'event'].unique()
        Znegative_events += len(bad_evt_ids)

        if bad_evt_ids.size > 0:
            df_sophronia = df_sophronia[~df_sophronia['event'].isin(bad_evt_ids)].copy()

        # Compute real Z position: using the drift velocity
        df_sophronia['Z_real'] = df_sophronia['Z'] * V_drift

        # Drop isolated clusters of hits (less than 4)
        data_event = df_sophronia.groupby('event')
        df_sophronia = data_event.apply(NO_hits_sp)
        df_sophronia = df_sophronia.reset_index(drop=True)

        # ----- Aggregated Sophronia Data ----- #
        # Aggregation functions
        def weighted_avg(series, weight):
            if weight.sum() == 0:                           # Avoid division by zero
                return np.nan
            return np.average(series, weights=weight)

        def R_max_func(group_df):
            return np.sqrt(group_df['X']**2 + group_df['Y']**2).max()
        
        # Relevant information from Sophronia: at event and npeak level
        df_file = df_sophronia.groupby(['event', 'npeak'], as_index=False).agg(

                        # Weighted averages for X, Y, Z
                        X=('X',      lambda x: weighted_avg(x, df_sophronia.loc[x.index, 'E_corr'])),
                        Y=('Y',      lambda y: weighted_avg(y, df_sophronia.loc[y.index, 'E_corr'])),
                        Z=('Z_real', lambda z: weighted_avg(z, df_sophronia.loc[z.index, 'E_corr'])),
                        # Sum of Ec
                        Ec=('E_corr', 'sum'),
                        # Min and max of Z
                        Z_min=('Z_real', 'min'),
                        Z_max=('Z_real', 'max'),
                        # Max R
                        # For R_max, the lambda needs the group DataFrame to access both X and Y
                        R_max=('X', lambda xy_group: R_max_func(df_sophronia.loc[xy_group.index]))

        )

        # ----- Dorothea ----- #
        df_dorothea = pd.DataFrame()
        try:
            df_dorothea = pd.read_hdf(h5_path, key=dorothea_key, columns=['event', 'nS1', 'nS2', 'S1h', 'S1e', 'DT'])
        except KeyError:
            print(f"Key {dorothea_key} not found in {h5_path}. Skipping...")
            continue


        # MATAR PO-LIKES PRIMERO

        # ----- S1e Correction ----- #
        df_dorothea = crudo.correct_S1e(df_dorothea, CV_fit, DT_stop, output_column='S1e_corr')

        # Relevant information from Dorothea: at event level
        dorothea_agg = df_dorothea.groupby('event', as_index=False).agg(

                            nS1=('nS1', 'max'),
                            nS2=('nS2', 'max'),
                            S1h=('S1h', 'max'),               # For events with > 1 S1, assign the max S1h
                            S1e_corr=('S1e_corr', 'max')      # Same here

        )

        # Merge aggregated Dorothea data with Sophronia data
        df_file = pd.merge(df_file, dorothea_agg, on='event', how='left')

        # ----- Run Information ----- #
        df_file['Run'] = run_id

        # Append to the main DataFrame
        all_processed_dfs.append(df_file)
        
        events += df_file.event.nunique()                # Per run
        final_reco_events += df_file.event.nunique()     # Total

    print(f"  → {events} reconstructed events\n")

print(f"\nTotal Sophronia events processed     = {sophronia_events}")
print(f"Total events with negative Z dropped = {Znegative_events}")
print(f"Total final events after processing  = {final_reco_events}")

--- Processing run 15502 ---
  → 24332 reconstructed events

--- Processing run 15504 ---


MemoryError: Unable to allocate 1.29 TiB for an array with shape (421125, 421125) and data type float64

In [19]:
# Concatenation
if all_processed_dfs:
    Data_df = pd.concat(all_processed_dfs, ignore_index=True)

print('Y ya, eso es todo, eso es todo')

Y ya, eso es todo, eso es todo


## Output

Let's store the final dataframe as an HDF5 file

In [20]:
# Output filename
h5_name = 'Data_event_background.h5'
h5_path = os.path.join(output_dir, h5_name)

# Store it!
Data_df.to_hdf(h5_path, key='/Data/Events', mode='w', format='table')
print(f"Data saved to {h5_path}")

Data saved to /lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Backgrounds/h5/Data_event_background.h5


Or... if you want to merge two different HDF5 files, you can do it ;)

In [21]:
# Paths of the HDF5 files
temp_h5_path = os.path.join(output_dir, 'temp.h5')
temp_df = pd.read_hdf(temp_h5_path, key='/Data/Events')

# Merge the DataFrames
merged_df = pd.concat([Data_df, temp_df], ignore_index=True)

# Output filename for the merged file
merged_h5_name = 'Merged_Data_event_background.h5'
merged_h5_path = os.path.join(output_dir, merged_h5_name)

# Store the merged DataFrame
merged_df.to_hdf(merged_h5_path, key='/Data/Events', mode='w', format='table')

# MC Production

First, write down all the basic information about the simulation files.

In [ ]:
# ----- Isotopes ----- #
Isotopes = ['Bi214', 'Tl208', 'Co60', 'K40']

## Processing

In [ ]:
# Store all the processed dfs
all_processed_dfs = []

# ----- Isotopes Loop ----- #
for isotope in Isotopes:

    print(f"\n--- Processing {isotope} ---\n")

    # Isotope directory and sub-folders (volumes)
    isotope_dir     = os.path.join(MC_dir, isotope)
    isotope_folders = [folder for folder in os.listdir(isotope_dir) if os.path.isdir(os.path.join(isotope_dir, folder))]
    print(f"Processing information in {len(isotope_folders)} volumes...")

    # ----- Volumes Loop ----- #
    for volume in isotope_folders:

        # Skip active volume. We do not expect background there.
        if volume == 'ACTIVE':
            continue

        # MC Sophronia path
        soph_dir = os.path.join(isotope_dir, volume, 'sophronia')
        # List all .h5 files in the MC Sophronia directory in alphabetical order
        files = sorted(glob.glob(os.path.join(soph_dir, '*.h5')))
        print(f"  {volume} volume: {len(files)} files")

        # Auxiliary variables
        reco_evts = 0

        # ----- Files Loop ----- #
        for file in files:

            # ----- Sophronia ----- #
            df_sophronia = pd.DataFrame()
            try:
                df_sophronia = pd.read_hdf(file, key=sophronia_key, columns=['event', 'npeak', 'X', 'Y', 'Z', 'Ec'])
            except KeyError:
                print(f"Key {sophronia_key} not found in {file}. Skipping...")
                continue

            # Set hits with NaN or negative energy to 0
            df_sophronia['Ec'] = np.where(
                                                pd.notna(df_sophronia['Ec']) & (df_sophronia['Ec'] > 0),        # Condition
                                                df_sophronia['Ec'],                                             # Value if condition is True
                                                0                                                               # Value if condition is False   
            )

            # Aggregation functions
            def weighted_avg(series, weight):
                if weight.sum() == 0:                           # Avoid division by zero
                    return np.nan
                return np.average(series, weights=weight)

            def R_max_func(group_df):
                return np.sqrt(group_df['X']**2 + group_df['Y']**2).max()
            
            # Relevant information from Sophronia: at event and npeak level
            df_file = df_sophronia.groupby(['event', 'npeak'], as_index=False).agg(

                            # Weighted averages for X, Y, Z
                            X=('X', lambda x: weighted_avg(x, df_sophronia.loc[x.index, 'Ec'])),
                            Y=('Y', lambda y: weighted_avg(y, df_sophronia.loc[y.index, 'Ec'])),
                            Z=('Z', lambda z: weighted_avg(z, df_sophronia.loc[z.index, 'Ec'])),
                            # Sum of Ec
                            Ec=('Ec', 'sum'),
                            # Min and max of Z
                            Z_min=('Z', 'min'),
                            Z_max=('Z', 'max'),
                            # Max R
                            # For R_max, the lambda needs the group DataFrame to access both X and Y
                            R_max=('X', lambda xy_group: R_max_func(df_sophronia.loc[xy_group.index]))

            )

            # ----- Dorothea ----- #
            df_dorothea = pd.DataFrame()
            try:
                df_dorothea = pd.read_hdf(file, key=dorothea_key, columns=['event', 'nS1', 'nS2', 'S1e'])
            except KeyError:
                print(f"Key {dorothea_key} not found in {file}. Skipping...")
                continue

            # Relevant information from Dorothea: at event level
            dorothea_agg = df_dorothea.groupby('event', as_index=False).agg(

                                nS1=('nS1', 'max'),
                                nS2=('nS2', 'max'),
                                S1e=('S1e', 'max')      # For events with > 1 S1, assign the max S1e

            )

            # Merge aggregated Dorothea data with Sophronia data
            df_file = pd.merge(df_file, dorothea_agg, on='event', how='left')

            # ----- Monte Carlo Information ----- #
            df_file['Isotope'] = isotope
            df_file['Volume']  = volume    

            # Append to the main DataFrame
            all_processed_dfs.append(df_file)
            reco_evts += df_file.event.nunique()

        print(f"  → {reco_evts} reco events processed")
            


--- Processing Bi214 ----

Processing information in 23 volumes...
  OPTICAL_PAD volume: 24 files
  → 22239 reco events processed
  GATE_RING volume: 3 files
  → 4661 reco events processed
  LIGHT_TUBE volume: 11 files
  → 51637 reco events processed
  PMT volume: 66 files
  → 45168 reco events processed
  EDPM_SEAL volume: 156 files
  → 1 reco events processed
  PEDESTAL volume: 1000 files
  → 611 reco events processed
  EP_COPPER_PLATE volume: 258 files
  → 35737 reco events processed
  DB_PLUG volume: 489 files
  → 1761 reco events processed
  TP_COPPER_PLATE volume: 7 files
  → 1403 reco events processed
  FIELD_RING volume: 88 files
  → 168820 reco events processed
  SAPPHIRE_WINDOW volume: 208 files
  → 198686 reco events processed
  SHIELDING_STEEL volume: 64 files
  → 196 reco events processed
  SHIELDING_STRUCT volume: 1000 files
  → 308 reco events processed
  BUBBLE_SEAL volume: 1000 files
  → 1 reco events processed
  CATHODE_RING volume: 3 files
  → 5541 reco events proce

In [36]:
# Concatenation
if all_processed_dfs:
    MC_df = pd.concat(all_processed_dfs, ignore_index=True)

print('Y ya, eso es todo, eso es todo')

Y ya, eso es todo, eso es todo


## Output

Let's store the final dataframe as an HDF5 file

In [ ]:
# Output filename
h5_name = 'MC_event_background.h5'
h5_path = os.path.join(output_dir, h5_name)

# Store it!
MC_df.to_hdf(h5_path, key='/MC/Events', mode='w', format='table')
print(f"Simulation saved to {h5_path}")

Output directory: /lustre/ific.uv.es/prj/gl/neutrinos/users/ccortesp/NEXT-100/Backgrounds/h5/
